This notebook contains some basic notes on how to use graph-tool, 
an advanced analytics package for in-memory graphs.
https://graph-tool.skewed.de/

In [44]:
from graph_tool.all import *
import numpy as np
import pandas as pd
import time

# Graphs, Vertices and Edges

The basic objects are 
1. Graphs
2. Vertices
3. Edges
We should first see how these things are constructed in graph-tool

## Vertices

In [2]:
# For simplicity we will init an undirected graph.
G = Graph(directed=False)

In [3]:
# We now add a single vertex
G.add_vertex()

<Vertex object with index '0' at 0x138a69750>

In [4]:
# This is one way to access an iterator over the vertices.
G.vertices()

In [5]:
# The elements are custom graph-tool classes called "vertices"
type(list(G.vertices())[0])

graph_tool.libgraph_tool_core.Vertex

In [6]:
# There is another way to access an iterator over the vertices,
# this produces just a list of integers, one-can think of it a bit like 
# a primary key for the vertices.
print(G.get_vertices())

[0]


In [7]:
# Now we add a number of vertices
N=5
G.add_vertex(N)

# and see that there these vertices were added to the graph 
# although so far our graph is just a collection of disconnected vertices.
print(G.get_vertices())

[0 1 2 3 4 5]


In [8]:
# In fact another way to access the same iterator as G.get_vertices 
# is through the vertex_index property map.
# More on property maps later...
print([G.vertex_index[vertex] for vertex in G.vertices()])

[0, 1, 2, 3, 4, 5]


## Edges

In [9]:
# We can add an edge
G.add_edge(3,5)

<Edge object with source '3' and target '5' at 0x138a45dc8>

In [10]:
# We can access an iterator for edges.
# Note that the simplest edge, consists of a triple:
# [vertex_1, vertex_2, edge_index].
# The edges also have an index (again, similar to a primary key for edges)
G.get_edges()[:,2:3]

array([[0]], dtype=uint64)

In [11]:
# We can also access an iterator over the collection of custom "edge" objects:
list(G.edges())

[<Edge object with source '3' and target '5' at 0x138a45d50>]

In [12]:
# And we can access an iterator just over the index
[G.edge_index[edge] for edge in G.edges()]

[0]

In [13]:
# Which for the current simple graph is essentially the same as 
G.get_edges()[:,2:3]

array([[0]], dtype=uint64)

# Edge-Lists

The most compact form of a graph is an **edge-list**

In [14]:
# This very simple edge-list says that 
# node_1 is connected to node_7
# node_7 is connected to node_4
# node_4 is connected to node_1

edge_list = [[1,7],[7,4],[4,1]]

# Indeed, this contains the defining information of a single connected graph.
# We do not need to seperately list the nodes, since we can 
# construct a list of nodes by forming the set of unique 
# nodes which have edges to them. 
# In this way we neglect isolated nodes, which may or may not be what
# your use case requires.

In [15]:
# We would like to load this directly into a graph, lets try the following:
G = Graph(directed=False)
G.add_edge_list(edge_list)

# However we see that something very odd has happened, there are more vertices
# than we anticipated. Our edge list has only three vertices [1,4,7] but 
# our graph now has vertices from 0-7 inclusive.
G.get_vertices()

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint64)

In [16]:
# Lets play with some keyword arguments in add_edge_list
G = Graph(directed=False)
G.add_edge_list(edge_list, hashed=True)

# We see that we have done a little better, there is now only three nodes,
# but they have been renumbered from zero and the order in which we should map
# them back to our original vertices is unclear.
G.get_vertices()

array([0, 1, 2], dtype=uint64)

In [17]:
# Graph-tool orders vertices by the order in which they have been loaded into the graph
# but still, this would need to be made more precise to allow us to construct a mapping between
# the two types of vertex-indices.

# To understand more deeply how to resolve the tension between these indices requires understanding
# property_maps

# Property Maps

Property maps are essential features of graph-tool,
they are used to assign properties to vertices and edges.

For example, if our vertices are people, a vertex_property could be gender, age, weight
and an edge_property might be "friend" or "enemy". 

If our nodes are geo-locations, a vertex_property might be "traffic_light" or "house"
and an edge_property might be "distance_in_meters".

The possibilities are boundless, properties are crucially important.


One strategy (used in another Python graph library NetworkX) would be to 
directly assign properties as attributes to the vertex-class or edge-class.
This adds bulk to vertices and edges, such that if during our analysis
we create some collection of vertex or edge classes, it will need memory for 
the whole class including the (perhaps numerous) attributes. 

The strategy employed by graph-tool is lighter: the properties of both edges and vertices
are attached as hash-tables to the graph itself, then the property of any particular
vertex or edge can be obtained in O(1) lookup time.

We now see how this works in more detail.

In [18]:
# We first instantiate a vertex-property on our graph,
# note that "vp" is an alias for vertex_properties.

G.vp["name"] = G.new_vertex_property("string")
print(G.vp)
print(G.vertex_properties)

{'name': <PropertyMap object with key type 'Vertex' and value type 'string', for Graph 0x138a713c8, at 0x138a71320>}
{'name': <PropertyMap object with key type 'Vertex' and value type 'string', for Graph 0x138a713c8, at 0x138a71320>}


In [19]:
# From the output we see that attached to the graph, is a dict "vertex_properties" 
# whose keys are the names of vertex_properties and whose values are custom PropertyMap objects.

In [20]:
# A PropertyMap also behaves like a dict whose keys are vertices.
# We first show that the value of each property map is the empty string, 
# then that we set the values to something rather canonical

print([G.vp['name'][vertex] for vertex in G.vertices()])

i=0
for vertex in G.vertices():
    G.vp['name'][vertex] = "vertex_number_{}".format(i)
    i+=1
    
print([G.vp['name'][vertex] for vertex in G.vertices()])

['', '', '']
['vertex_number_0', 'vertex_number_1', 'vertex_number_2']


In [21]:
# Edge properties are added in a similar manner.
# Here we add an edge_property with the name "distance" and since distances are usually
# decimals, we set its value to a double (or float).

G.ep["distance"] = G.new_edge_property("double")
print(G.ep)
print(G.edge_properties)

{'distance': <PropertyMap object with key type 'Edge' and value type 'double', for Graph 0x138a713c8, at 0x138aed7f0>}
{'distance': <PropertyMap object with key type 'Edge' and value type 'double', for Graph 0x138a713c8, at 0x138aed7f0>}


## Resolving the vertex_index conundrum with PropertyMaps

In [22]:
# We can now resolve the conundrum we had where graph-tool 
# assigns a particular (sequential) index to a vertex 
# while we may have had a different index in our edge_list

# I find that the following method is accurate if somewhat inelegant 
# in that we have to write to use pandas and write to disk

df = pd.DataFrame.from_records(edge_list, columns=['node_1', 'node_2'])
df.to_csv('edge_list.csv', sep=',', index=False)
G = load_graph_from_csv('edge_list.csv', skip_first=True, directed=False, hashed=True)

print("vertices: ", G.get_vertices())
print("vertex_properties: ", G.vp.keys())

vertices:  [0 1 2]
vertex_properties:  ['name']


In [23]:
# We see that load_graph_from_csv with hashed=True has created a vertex_property for us called "name"
# The value of this property is our original index for the vertex:

print([index for index in G.get_vertices()])
print([G.vp['name'][vertex] for vertex in G.vertices()])

[0, 1, 2]
['1', '7', '4']


In [24]:
# So this way we can construct a map from our old index to the graph_tool index and back

gt_us_dict = {G.vertex_index[vertex]: G.vp['name'][vertex] for vertex in G.vertices()}
us_gt_dict = {G.vp['name'][vertex]:G.vertex_index[vertex] for vertex in G.vertices()}

In [25]:
print(gt_us_dict)
print(us_gt_dict)

{0: '1', 1: '7', 2: '4'}
{'1': 0, '7': 1, '4': 2}


In [26]:
# We can also have a dict whose keys are our vertex_indices and who vals are the vertex_class objects:
vertex_dict = {G.vp['name'][vertex]:vertex for vertex in G.vertices()}
print(vertex_dict)

{'1': <Vertex object with index '0' at 0x138a69db0>, '7': <Vertex object with index '1' at 0x138a69ed0>, '4': <Vertex object with index '2' at 0x138a69f30>}


# Input-Output

In [27]:
# While we saw that an edge-list is a minimal way to present simple graphs 
# and that this can easily be written to a csv, 
# once one has instantiated various property maps, 
# it's best to save a graph-tool object as a custom binary .gt file.

In [28]:
G.save('graph.gt', fmt='gt')
G.load('graph.gt')

In [30]:
G.vp

{'name': <PropertyMap object with key type 'Vertex' and value type 'string', for Graph 0x138aedc18, at 0x138aed710>}

## Performance

In [56]:
# To test the performance we load a 150Mb graph from facebook
# https://www.kaggle.com/c/FacebookRecruiting

df = pd.read_csv('fbook_edge_list.csv')
vertices_unique = set(list(df['source_node'].unique()) + list(df['destination_node'].unique())) 
print("Number of unique nodes: {}".format(len(vertices_unique)))
df[:5]

Number of unique vertices: 1862220


,source_node,destination_node
0,1,690569
1,1,315892
2,1,189226
3,2,834328
4,2,1615927


In [51]:
# For loading from a csv I find: 26.3s

start = time.time()
G = load_graph_from_csv('fbook_edge_list.csv', skip_first=True, directed=True, hashed=True)
print(time.time() - start)

27.242825031280518


In [57]:
print("Number of vertices: {}".format(len(G.get_vertices())))

Number of vertices: 1862220


In [46]:
# Saveing to a .gt file is about half a second
%timeit -r 1 G.save('fbook_edge_list.gt', fmt='gt')

505 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [47]:
# Loading from the .gt file is around 3seconds.
G = Graph()
start = time.time()
G.load('fbook_edge_list.gt', fmt='gt')
print(time.time() - start)

2.860599994659424


In [59]:
# Checking that our graph is indeed what we think it is
len(G.get_vertices())

1862220

# Shortest Distance

In [ ]:
# An important concept in graph-theory is that of a shortest path.
# Computationaly, it is the same to compute the shortest path and the shortest_distance.

In [106]:
G.load('fbook_edge_list.gt', fmt='gt')

In [108]:
# 1. select pairs of vertices at random
# 2. compute the shortest distance between pair-nodes

# The time seems to be a bit less than 100ms per shortest_distance computation 

num_pairs = 50
random_pairs = np.random.randint(low=0, high=G.get_vertices().max(), size=(num_pairs, 2))
%timeit -r 1 [shortest_distance(G, G.vertex(pair[0]),G.vertex(pair[1])) for pair in random_pairs]

6.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Shortest Path with weighted edges

In [89]:
# We first add a weigthed edge to the facebook graph

df = pd.read_csv('fbook_edge_list.csv')
lengths = np.random.randint(low=0, high=1000000, size=(len(df), ))
df['length'] = lengths

start = time.time()
df.to_csv('fbook_edge_list_lengths.csv', sep=',', index=False)
print("write to csv: {}".format(time.time() - start))

write to csv: 


In [109]:
# Here we have used certain kw-arguments 
# to correctly parse the edge-PropertyMap:
# eprop_types = ['int'] 
# eprop_names=['length']

start = time.time()
G_w = load_graph_from_csv('fbook_edge_list_lengths.csv', 
                        skip_first=True, 
                        directed=True, 
                        hashed=True, 
                        eprop_types = ['int'],
                        eprop_names=['length'])

print("load from csv: {}".format(time.time() - start))

load from csv: 34.16386294364929


In [114]:
# We see here that now that the paths are weighted
# note the kw-argument weights=G_w.ep['length'] in shortest_distance

# each shortest_distance calculation now takes around 1 second.
# This is an increase of a factor of about ten from the unweighted computation

num_pairs = 10
random_pairs = np.random.randint(low=0, high=G_w.get_vertices().max(), size=(num_pairs, 2))

%timeit -r 1 [shortest_distance(G_w, G_w.vertex(pair[0]),G_w.vertex(pair[1]), \
                                weights=G_w.ep['length']) for pair in random_pairs]

43.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Graph Centrality measures

In [117]:
# The pagerank algorithm converges in undere 20seconds
# when we ignore the "length" PropertyMap

# pagerank returns an external PropertyMap, it is external in that it
# is not directly a property of the graph.

start = time.time()
pr = pagerank(G_w)
print("Time to compute pagerank of our unweighted graph: {}s".format(time.time() - start))

Time to compute pagerank of our unweighterd graph: 18.162451028823853


In [128]:
# The pagerank algorithm converges in just over 20seconds
# when we include the "length" PropertyMap

start = time.time()
pr_w = pagerank(G_w, weight = G_w.ep['length'])
print("Time to compute pagerank of our weighted graph: {}s".format(time.time() - start))

Time to compute pagerank of our weighted graph: 27.264307022094727


In [126]:
vertices_pr = [pr[vertex] for vertex in G_w.vertices()]
vertices_pr[:5]

[2.7844607326125795e-07,
 1.6461054646314429e-06,
 1.6770951207320375e-06,
 4.902623902062898e-07,
 3.7713027051676735e-07]

In [129]:
vertices_pr_w = [pr_w[vertex] for vertex in G_w.vertices()]
vertices_pr_w[:5]

[2.7844607326125795e-07,
 1.6461054646314429e-06,
 1.6770951207320375e-06,
 4.902623902062898e-07,
 3.7713027051676735e-07]

# Neighbours

In [134]:
num_nodes = 10000
random_nodes = np.random.randint(low=0, high=G_w.get_vertices().max(), size=(num_nodes, ))

start = time.time()
all_nbrs = [vertex.all_neighbors() for vertex in G.vertices()]
print("Time to compute all_neighbours per vertex: {} ms".format((time.time() - start)/num_nodes*10**3))

Time to compute all_neighbours per vertex: 0.5709533929824829 ms


In [135]:
G_w.get_vertices().max()

1862219